In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_0_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_0_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_0_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True
                     )


In [5]:
model_name = 'rgcn_0_transductive'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200

In [8]:
model = RGCN(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameters, but no reset_parameters.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameter

Number of parameters: 2038872
Space occupied: 8155488 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory rgcn_0_transductive created successfully!


In [10]:
learning_rate = 5e-2 #1e-3 stops after 20 iterations and actually does not reduce the error too much
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Using RGCN without graph-based sampling! Please select sampler="schlichtkrull" instead of None.
Using RGCN without graph-based sampling! Please select sampler="schlichtkrull" instead of None.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameters, but no reset_parameters.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedd

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 4.497657299041748
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.000767058415083916
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.00014274667258523557
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.00045490254383457566
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0007670582272112371
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.00014274667773861438
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0004549024743027985
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0007670581399773549
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00014274667258523557
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.00045490240628129526
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validation.h

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

...

Step: 24
Metric: loss = 3.4063024520874023
Step: 24
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.03406047119975361
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.011245190585236896
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.022652830892495254
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.03406047075986862
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.011245191097259521
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.02265283092856407
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.03406047108418372
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.01124519057263418
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.022652830828408948
Metric: validation.head.optimistic.hits_at_1 = 0.014396887159533073
Metric: validation.tail.optimistic.hits_at_1 = 0.003501945525291829
Metric: validation.both.optimistic.hits_at_1 = 0.008

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 25
Metric: loss = 2.493467330932617
Step: 25
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.04119685404285297
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.014839359190812421
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.028018106616832694
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.041196852922439575
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.014839360490441322
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.028018105775117874
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.041196853950309156
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.014839359190812421
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.02801810657056079
Metric: validation.head.optimistic.hits_at_1 = 0.015175097276264591
Metric: validation.tail.optimistic.hits_at_1 = 0.005447470817120622
Metric: validation.both.optimistic.hits_at_1 = 0.0

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 26
Metric: loss = 2.2845022678375244
Step: 26
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0438120540198866
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.01446363247533221
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.029137843247609405
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.04381205886602402
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.014463631436228752
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.029137846082448956
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.043812053976290156
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.01446363246578041
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.02913784322103528
Metric: validation.head.optimistic.hits_at_1 = 0.015175097276264591
Metric: validation.tail.optimistic.hits_at_1 = 0.004280155642023347
Metric: validation.both.optimistic.hits_at_1 = 0.0097

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0    4.497657
1   12.534678
2    3.945479
3    7.577858
4    9.555771
5    9.027675
6    6.876003
7    6.677511
8    7.442837
9    6.857717
10   4.859990
11   2.701027
12   4.540019
13   5.295308
14   4.163463
15   3.483821
16   3.256724
17   3.731028
18   4.112164
19   3.734639
20   2.878115
21   2.346889
22   3.162326
23   3.406302
24   2.493467
25   2.284502


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/10.4k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.017567   0.017567     0.017567
hits_at_1                     0.005298   0.005298     0.005298
hits_at_3                     0.015605   0.015605     0.015605
hits_at_5                     0.024949   0.024949     0.024949
hits_at_10                    0.037761   0.037761     0.037761
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.010759   0.010759     0.010759
hits_at_1                     0.001638   0.001638     0.001638
hits_at_3                     0.006069   0.006069     0.006069
hits_at_5                     0.009248   0.009248     0.009248
hits_at_10                    0.015124   0.015124     0.015124
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.014163   0.014163     0.014163
hits_at_1                     0.003468   0.003468     0.003468
hits_at_3                     0.010837  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/2.57k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.047954   0.047954     0.047954
hits_at_1                     0.016342   0.016342     0.016342
hits_at_3                     0.047860   0.047860     0.047860
hits_at_5                     0.072374   0.072374     0.072374
hits_at_10                    0.101556   0.101556     0.101556
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.019475   0.019475     0.019475
hits_at_1                     0.006615   0.006615     0.006615
hits_at_3                     0.014786   0.014786     0.014786
hits_at_5                     0.018677   0.018677     0.018677
hits_at_10                    0.028794   0.028794     0.028794
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.033715   0.033715     0.033715
hits_at_1                     0.011479   0.011479     0.011479
hits_at_3                     0.031323  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/3.27k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.062925   0.062925     0.062925
hits_at_1                     0.030581   0.030581     0.030581
hits_at_3                     0.062691   0.062691     0.062691
hits_at_5                     0.089602   0.089602     0.089602
hits_at_10                    0.111621   0.111621     0.111621
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.020073   0.020073     0.020073
hits_at_1                     0.006728   0.006728     0.006728
hits_at_3                     0.014679   0.014679     0.014679
hits_at_5                     0.020183   0.020183     0.020183
hits_at_10                    0.031193   0.031193     0.031193
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.041499   0.041499     0.041499
hits_at_1                     0.018654   0.018654     0.018654
hits_at_3                     0.038685  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   

                                                value  
0                                              cuda:0  
1                                             81554

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)